In [ ]:
from google.colab import drive
drive.mount('/content/drive')

colabPath = 'drive/My Drive/Colab Notebooks/AIAP16/'

In [ ]:
dataset_path = colabPath + "lung_cancer.db"

In [ ]:
# import libraries
import sqlite3 as sq3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Connect to database
conn = sq3.Connection(dataset_path)

In [ ]:
# Create query statement
query = """
SELECT *
FROM lung_cancer;
"""

In [ ]:
# Load data into Dataframe and view first 5 rows of db
df = pd.read_sql(query, conn)
df.head()

In [ ]:
# check the shape of the dataframe
df.shape

In [ ]:
df.describe()

I need to drop the ID column as it is not meaningful and thus, not used for analysis.

In [ ]:
df = df.drop('ID', axis=1)
df.head()

The minimum age as shown in the summary of the data, cannot be negative. I will investigate that now.

In [ ]:
# Count the sum of values in Age column that are less than 0
age_less_than_zero = (df['Age'] < 0).sum()
print(age_less_than_zero)

In [ ]:
filtered_age_df = df[(df['Age'] < 0)]
filtered_age_df.head()

After investigations, I choose to replace the negative values as it is but make it positive. Reason for that is that I assume that it was a typo error or system error during input.

In [ ]:
# Replace the negative values as positive
df['Age'] = df['Age'].abs()
df.head()

In [ ]:
df.describe()

I want to view the distribution of the sample size of Lung Cancer Occurence by age

In [ ]:
# Define age bins and labels
bins = [0, 18, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-110', '111-120']

# Create Age Group column based on Age column
df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

df.head()

In [ ]:
sns.countplot(x='Age Group', hue='Lung Cancer Occurrence', data=df)
plt.title('Lung Cancer Occurrence by Age')
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.show()

I need to encode the data in the columns of COPD History, Genetic Markers, Air Pollution Exposure, Taken Bronchodilators, Frequency of Tiredness and Dominant Hand

In [ ]:
# Check the categories in Gender
unique_gender = df['Gender'].unique()
print(unique_gender)

I need to check the number of NAN values in Gender column

In [ ]:
# Count the number of NaN values
nan_count = df['Gender'].isna().sum()
print(f'Number of NaN values in the Gender column: {nan_count}')

It seems that NAN is a string. I shall check the number of NAN values.

In [ ]:
matching_count = len(df[df['Gender'] == 'NAN'])
print(f'Number of NAN values in the Gender column: {matching_count}')

Since None value in Gender column is only 1 row, I shall drop the row.

In [ ]:
# Drop rows where Gender is equivalent to NAN and check the shape again
df = df.loc[df['Gender'] != 'NAN']
df.shape

I shall change all the text in Gender column to uppercase

In [ ]:
#Change values in Gender column to uppercase
df['Gender'] = df['Gender'].str.upper()

In [ ]:
# Check the categories in Gender
unique_gender = df['Gender'].unique()
print(unique_gender)

I want to view the distribution of the sample size of Lung Cancer Occurence by Gender

In [ ]:
sns.countplot(x='Gender', hue='Lung Cancer Occurrence', data=df)
plt.title('Lung Cancer Occurrence by Gender')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()

I shall perform one hot encoding on the dataset

In [ ]:
# Apply one-hot encoding
gender_encoded = pd.get_dummies(df['Gender'], prefix='Gender')
print(gender_encoded)

In [ ]:
# Add encoded columns to dataset
df = pd.concat([df, gender_encoded], axis=1)
df.head()

In [ ]:
# Check the categories in COPD History
unique_cop_hist = df['COPD History'].unique()
print(unique_cop_hist)

In [ ]:
# Count the number of None values
none_count = df['COPD History'].isnull().sum()
print(f'Number of None values in the COPD History column: {none_count}')

As the None values in the COPD History is about 10.7% of the dataset (1112/10348), I will choose to fill it as another flag.

In [ ]:
# Fill the None values in COPD_History column with the string 'unknown'.
df['COPD History'] = df['COPD History'].fillna('unknown')
# Print the unique values again to check
unique_cop_hist = df['COPD History'].unique()
print(unique_cop_hist)

I want to view the distribution of the sample size of Lung Cancer Occurence by COPD History

In [ ]:
sns.countplot(x='COPD History', hue='Lung Cancer Occurrence', data=df)
plt.title('Lung Cancer Occurrence by COPD History')
plt.xlabel('COPD History')
plt.ylabel('Count')
plt.show()

In [ ]:
# Apply one-hot encoding
copd_hist_encoded = pd.get_dummies(df['COPD History'], prefix='COPD History')
print(copd_hist_encoded)

In [ ]:
# Add encoded columns to dataset
df = pd.concat([df, copd_hist_encoded], axis=1)
df.head()

In [ ]:
# Check the categories in Genetic Markers
unique_genetics_markers = df['Genetic Markers'].unique()
print(unique_genetics_markers)

I want to view the distribution of the sample size of Lung Cancer Occurence by Genetic Markers

In [ ]:
sns.countplot(x='Genetic Markers', hue='Lung Cancer Occurrence', data=df)
plt.title('Lung Cancer Occurrence by Genetic Markers')
plt.xlabel('Genetic Markers')
plt.ylabel('Count')
plt.show()

In [ ]:
# Apply one-hot encoding
genetics_markers_encoded = pd.get_dummies(df['Genetic Markers'], prefix='Genetic Markers')
print(genetics_markers_encoded)

In [ ]:
# Add encoded columns to dataset
df = pd.concat([df, genetics_markers_encoded], axis=1)
df.head()

In [ ]:
# Check the categories in Air Pollution Exposure
unique_air_pol_exp = df['Air Pollution Exposure'].unique()
print(unique_air_pol_exp)

In [ ]:
# Count the number of None values in Air Pollution Exposure column
none_count = df['Air Pollution Exposure'].isnull().sum()
print(f'Number of None values in the Air Pollution Exposure column: {none_count}')

Since None value in Air Pollution Exposure column is only 3 rows, I shall drop the rows.

In [ ]:
# Check shape before dropping
df.shape

In [ ]:
# Drop rows where Gender is equivalent to None and check the shape again
df = df.dropna(subset=['Air Pollution Exposure'])
df.shape

I want to view the distribution of the sample size of Lung Cancer Occurence by Air Pollution Exposure

In [ ]:
sns.countplot(x='Air Pollution Exposure', hue='Lung Cancer Occurrence', data=df)
plt.title('Lung Cancer Occurrence by Air Pollution Exposure')
plt.xlabel('Air Pollution Exposure')
plt.ylabel('Count')
plt.show()

In [ ]:
# Apply one-hot encoding
air_pol_exp_encoded = pd.get_dummies(df['Air Pollution Exposure'], prefix='Air Pollution Exposure')
print(air_pol_exp_encoded)

In [ ]:

# Add encoded columns to dataset
df = pd.concat([df, air_pol_exp_encoded], axis=1)
df.head()

In [ ]:
# Check the categories in Taken Bronchodilators
unique_taken_broncho = df['Taken Bronchodilators'].unique()
print(unique_taken_broncho)

In [ ]:
# Count the number of None values in Taken Bronchodilators column
none_count = df['Taken Bronchodilators'].isnull().sum()
print(f'Number of None values in the Taken Bronchodilators column: {none_count}')

As the None values in the Taken Bronchodilators is about 10.2% of the remaining dataset (1060/10344), I will choose to fill it as another flag.

In [ ]:
# Fill the None values in Taken Bronchodilators column with the string 'unknown'.
df['Taken Bronchodilators'] = df['Taken Bronchodilators'].fillna('unknown')
# Print the unique values again to check
unique_taken_broncho = df['Taken Bronchodilators'].unique()
print(unique_taken_broncho)

I want to view the distribution of the sample size of Lung Cancer Occurence by Taken Bronchodilators

In [ ]:
sns.countplot(x='Taken Bronchodilators', hue='Lung Cancer Occurrence', data=df)
plt.title('Lung Cancer Occurrence by Taken Bronchodilators')
plt.xlabel('Taken Bronchodilators')
plt.ylabel('Count')
plt.show()

In [ ]:
# Apply one-hot encoding
taken_broncho_encoded = pd.get_dummies(df['Taken Bronchodilators'], prefix='Taken Bronchodilators')
print(taken_broncho_encoded)

In [ ]:
# Add encoded columns to dataset
df = pd.concat([df, taken_broncho_encoded], axis=1)
df.head()

In [ ]:
# Check the categories in Frequency of Tiredness
unique_freq_tired = df['Frequency of Tiredness'].unique()
print(unique_freq_tired)

I want to view the distribution of the sample size of Lung Cancer Occurence by Frequency of Tiredness

In [ ]:
sns.countplot(x='Frequency of Tiredness', hue='Lung Cancer Occurrence', data=df)
plt.title('Lung Cancer Occurrence by Frequency of Tiredness')
plt.xlabel('Frequency of Tiredness')
plt.ylabel('Count')
plt.show()

In [ ]:
# Apply one-hot encoding
freq_tired_encoded = pd.get_dummies(df['Frequency of Tiredness'], prefix='Frequency of Tiredness')
print(freq_tired_encoded)

In [ ]:
# Add encoded columns to dataset
df = pd.concat([df, freq_tired_encoded], axis=1)
df.head()

In [ ]:
# Check the categories in Dominant Hand
unique_dom_hand = df['Dominant Hand'].unique()
print(unique_dom_hand)

I want to view the distribution of the sample size of Lung Cancer Occurence by Dominant Hand

In [ ]:
sns.countplot(x='Dominant Hand', hue='Lung Cancer Occurrence', data=df)
plt.title('Lung Cancer Occurrence by Dominant Hand')
plt.xlabel('Dominant Hand')
plt.ylabel('Count')
plt.show()

In [ ]:
# Apply one-hot encoding
dom_hand_encoded = pd.get_dummies(df['Dominant Hand'], prefix='Dominant Hand')
print(dom_hand_encoded)

In [ ]:
# Add encoded columns to dataset
df = pd.concat([df, dom_hand_encoded], axis=1)
df.head()

Now I will need to calculate the difference of Last Weight and Current Weight columns to derive a new column named Weight Change.

In [ ]:
# Count the number of None values in Last Weight column
none_count = df['Last Weight'].isnull().sum()
print(f'Number of None values in the Last Weight column: {none_count}')

In [ ]:
# Count the number of NaN values in Last Weight column
nan_count = df['Last Weight'].isna().sum()
print(f'Number of NaN values in the Last Weight column: {nan_count}')

In [ ]:
# Count the number of None values in Current Weight column
none_count = df['Current Weight'].isnull().sum()
print(f'Number of None values in the Current Weight column: {none_count}')

In [ ]:
# Count the number of NaN values in Current Weight column
nan_count = df['Current Weight'].isna().sum()
print(f'Number of NaN values in the Current Weight column: {nan_count}')

In [ ]:
# Minus the Last Weight from Current Weight and put the result in a new column
df['Weight Change'] = df['Last Weight'] - df['Current Weight']
df.head()

In [ ]:
# Drop the Last Weight and Current Weight columns
df = df.drop(['Last Weight','Current Weight'], axis=1)
df.head()

In [ ]:
df.describe()

I want to view the distribution of the sample size of Lung Cancer Occurence by Weight Change

In [ ]:
# Define age bins and labels
bins = [-20, -15, -10, -5, 0, 5, 10, 15, 20]
labels = ['-20 to -16', '-15 to -11', '-10 to -6', '-5 to -1', '0 to 4', '5 to 9', '10 to 14', '15 to 20']

# Create Age Group column based on Age column
df['Weight Change Group'] = pd.cut(df['Weight Change'], bins=bins, labels=labels, right=False)

df.head()

I want to view the distribution of the sample size of Lung Cancer Occurence by Weight Change Group

In [ ]:
sns.countplot(x='Weight Change Group', hue='Lung Cancer Occurrence', data=df)
plt.title('Lung Cancer Occurrence by Weight Change Group')
plt.xlabel('Weight Change Group')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Now I will need to calculate the difference of 'Start Smoking' and 'Stop Smoking' columns to derive a new column named 'Years Quitted Smoking'.

In [ ]:
# Check the unique values in Start Smoking
unique_start_smoking = df['Start Smoking'].unique()
print(unique_start_smoking)

In [ ]:
# Check the unique values in Stop Smoking
unique_stop_smoking = df['Stop Smoking'].unique()
print(unique_stop_smoking)

It seems that 'Not Applicable' is for non smokers but I must investigate further. I will check if all. the rows in Start Smoking is Not Applicable has a corresponding value of the same in Stop Smoking column.

In [ ]:
# Check for different values where Start Smoking is 'Not Applicable' but Stop Smoking is not
filtered_df = df[(df['Start Smoking'] == 'Not Applicable') & (df['Stop Smoking'] != 'Not Applicable')]
print(filtered_df)

In [ ]:
# Check for different values where Stop Smoking is 'Not Applicable' but Start Smoking is not
filtered_df = df[(df['Start Smoking'] != 'Not Applicable') & (df['Stop Smoking'] == 'Not Applicable')]
print(filtered_df)

Since both Start Smoking and Stop Smoking have the same values where the value is 'Not Applicable', I can safely assume that 'Not Applicable' are for non-smokers. I will replace the Start Smoking column with values 'Not Applicable' with 0000 and Stop Smoking column with values 'Not Applicable' with 9999

In [ ]:
# Replace the values 'Not Applicable' with 0000
df['Start Smoking'] = df['Start Smoking'].replace('Not Applicable', '0000')
df['Stop Smoking'] = df['Stop Smoking'].replace('Not Applicable', '9999')
# Check the unique values in Start Smoking
unique_start_smoking = df['Start Smoking'].unique()
print(unique_start_smoking)

In [ ]:
# Check the unique values in Stop Smoking
unique_stop_smoking = df['Stop Smoking'].unique()
print(unique_stop_smoking)

I will now replace Still Smoking in Stop Smoking column with 9999

In [ ]:
# Replace the values 'Still Smoking' with 9999
df['Stop Smoking'] = df['Stop Smoking'].replace('Still Smoking', '9999')
# Check the unique values in Stop Smoking
unique_stop_smoking = df['Stop Smoking'].unique()
print(unique_stop_smoking)

Now I will deduct the values in Stop Smoking Column from Start Smoking column.  Any value with the value of 0 can be interpreted as a non-smoker. Any value above 3000, I can assume that the person is still smoking. Any negative value needs to be investigated further.

In [ ]:
# Change the datatype of values in Start Smoking and Stop Smoking to integers
df['Start Smoking'] = df['Start Smoking'].astype(int)
df['Stop Smoking'] = df['Stop Smoking'].astype(int)
df.head()

In [ ]:
# Subtract 'Stop Smoking' from 'Start Smoking' and store the result in a new column 'Years Quitted Smoking'
df['Years Quitted Smoking'] = df['Stop Smoking'] - df['Start Smoking']
df.head()

In [ ]:
# Replace the values in Years Quitted Smoking above 3000 with 9999
df.loc[(df['Years Quitted Smoking'] > 3000) & (df['Years Quitted Smoking'] != 9999), 'Years Quitted Smoking'] = 0
df.head()

In [ ]:
df.describe()

Since that there are no negative values in Years Quitted Smoking column, I can assume that it does not require further investigation.

In [ ]:
df.shape

In [ ]:
# Export the dataframe to csv
df.to_csv('cleaned_data.csv')

Now that I have cleaned and transformed the data, I will analyse it.

In [ ]:
# Calculate correlation
correlation_matrix = df.corr()

In [ ]:
# Create the heatmap
plt.figure(figsize=(20, 8))  # Size of the figure
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

# Add titles and labels if needed
plt.title('Correlation Matrix Heatmap')
plt.xlabel('Variables')
plt.ylabel('Variables')
plt.tight_layout()

# Show the plot
plt.show()

It seems that Weight Change has the strongest correlation to Lung Cancer Occurence. Most of the variables have very weak correlations.